# Implementing The MENACE Player

### Tracking MENACE's Beads

The original MATCHBOX-based MENACE remembered the lessons from past games using matchboxes and beads.
1. It used one matchbox for each board position that MENACE might enounter.
1. Each machbox was labelled. The label showed the position represented by the box, and showed each possible move from that position. Possible moves were shown in different colours.
1. Each matchbox contained beads of colours that matched those on the label. The beads determined the probability that MENACE would make the corresponding move.

In our implementation we will maintain the bead information as a two element vector, with each element an array.

The first element will be a vector of decoded (numerical) positions.
The second element will be a matrix of bead counts, with one row per possible position and 9 columns.

Each column will corespond to one of the possible moves from the position, and will contain the number of virtual beads corresponding to that move. Some moves will not be possible because the sqare is already occupied. We will initialse the bead counts for such positions to zero, and the counts will never get updated, since the correponding moves will never be made.

TODO: Add illustration of a position (in vector and display form) and its bead counts

The values for possible moves will be initialised to some fixed number when a new MENACE configuration is created, and will then get adjusted at the end of each game by MENACE's learning rule.

The bead data will form a new third element of the MENACE configuration.

### MENACE's Learning Rule

At the end of a game MENACE adjusted the bead counts.
1. If MENACE *won* it added three beads of the same colour matching those picked during the game.
1. If MENACE *drew* it added one bead of the appropriate colour to each matchbox used.
1. If MENACE *lost* it removed three beads of the appropriate colour from each matchbox used.

We'll do the same to our virtual bead counts.

Let's start by copying in our earlier work.

In [1]:
)copy notebook7

done

./notebook7.dws saved Mon Dec 21 09:30:16 2020

In [2]:
⎕io ← 0

## Simplifying MENACE's Approach

In his original design for MENACE, Donald Michie eliminated redundant board positions from consideration and specified that MENACE always made the first move.

He did this in order to reduce the number of machboxes required to a manageable number. A secondary benefit was that one game could provide reinforement for a whole family of symetric versions. 

Since we are representing the matchboxed by values in an array we are not constrained in the same way, and the time taken to train the system is also less of a problem.

We'll also allow a `menace_player` to be the first or second player.

We'll need more code to match a real game position to its canonical representation, so we won't do so in this chapter. We could maintain a virtual moatchbox for beads for every  possible position, but many of those could never occur in play. We can reduce the 19683 positions considerably by using the `ok` function which checks the number of `×`s and `○`s.

Let's see how many positions are `ok`.

In [3]:
all ← encode numbers← ⍳3*9 ⍝ all possible positions
ok←{0 1∊⍨-⌿+/1 2∘.=⍵}      ⍝ do positions have counts of × and ○ that could occur in play?
good ← (ok all)/numbers    ⍝ all positions are *ok* by that defintion
⍴good                      ⍝ how many positions are ok?

6046

That would be a lot of matchboxes, but an APL array of that size is not a problem on a modern computer.

We'll maintain virtual bean counts for each position in `good`

The bead data is a two-element vector.

The first is a `good` - a vector of decoded positions. We saved that earlier in a variable `ucpn`.

The second is a matrix of bead counts, with one row for each position in `good`, and nine columns corrspnding to the nine squares on the board. Each column shoud contain the initial bean count for those positions that are empty, and zero for each position that has already been filled.

Let's look at how to initialise the virtual bead data. MENACE started with 20 beads in each matchbox, but we will make that a changeable parameter called starting_count.

In [4]:
starting_count ← 20
bc ← ((⍴good),9)⍴ starting_count

Now we need to set the bead counts to zero for each position that has been filled.

We'll do that in stages. First we'll convert encode values in `ucpn`, That will give us a matrix `ucp` with one row per position and one column per board position.

Next we'll look for the non-zeros in `ucp`, keep the corresponding positions of bead counts unchanged, and set the rest to zero.

In [5]:
gbp ← encode good ⍝ a matrix with one board position per row
zero ← 0=gbp ⍝ a matrix with a 1 for each non-empty position and a 0 for every empty one.
bc ← bc × zero

In [6]:
list 5↑gbp

┌───┬───┬───┬───┬───┐
│...│...│...│...│...│
│...│...│...│...│...│
│...│..×│.×.│.×○│.○×│
└───┴───┴───┴───┴───┘

In [7]:
5↑bc

20 20 20 20 20 20 20 20 20
20 20 20 20 20 20 20 20 0
20 20 20 20 20 20 20 0 20
20 20 20 20 20 20 20 0 0
20 20 20 20 20 20 20 0 0

That looks correct. Let's turn that into a function `initial_counts` which will take the good array is its left argument and the number of intial counts as its right argument. 

In [8]:
initial_counts ← { bc ← ((⍴⍺),9)⍴ ⍵ ⋄ bc × 0=encode ⍺}

In [9]:
⍴good initial_counts 20

6046 9

Now we can easily create the extended initial configuration.

The third element contains `good` and the intial counts.

We'll write a function called `initialise` which will create the initial configuration.

In [10]:
initialise ← {(1 9⍴0) ⍬ ((⍺) (⍺ initial_counts ⍵))}
config ← good initialise 20

Now we can start work on the implementation of `menace_player`.

Like the `random_player` our `menace_player` function should expect a menace configuration as its argument, work out how to move, and return an updated configuration as its result.

It will work out which move to make by looking at the bead counts for each move it could make from the current position and chosing one at random. The probablilty of each possible move should reflect the proportion of beads allocated to that move.

Let's start by writing a function that will pick a move index given a vector of bead counts.

In [11]:
beads ← 2 0 1 0 3
pick ← {(0,+\⍵)⍸?+/⍵}

In [12]:
picks ← +/(⍳6)∘.=⍎¨1000⍴⊂'pick beads'
picks÷+/picks

0.337 0 0.171 0 0.492 0

In [13]:
cfp ← {good counts ← 2⊃⍺ ⋄ counts[good⍳decode ⍵;] }
menace_player ← {cf ← ⍵ ⋄ g ← cg ⍵ ⋄ cp ← p g ⋄ cp[pick cf cfp cp] ← np cp ⋄ g ← g⍪cp ⋄ cf ucg g }
config ← menace_player config

In [14]:
2↑config ← menace_player game_runner random_player⊢config

┌─────────────────┬──┐
│0 0 0 0 0 0 0 0 0│¯1│
│0 0 0 0 1 0 0 0 0│ │
│0 0 0 0 1 0 0 2 0│ │
│0 0 1 0 1 0 0 2 0│ │
│2 0 1 0 1 0 0 2 0│ │
│2 1 1 0 1 0 0 2 0│ │
│2 1 1 0 1 0 0 2 2│ │
│2 1 1 0 1 1 0 2 2│ │
│2 1 1 0 1 1 2 2 2│ │
└─────────────────┴──┘

In [15]:
1⊃config ← {menace_player game_runner random_player⊢⍵}⍣100⊢config

¯1 0 1 0 1 ¯1 0 0 1 0 1 1 ¯1 0 ¯1 0 0 0 1 1 0 1 1 1 0 0 1 ¯1 1 ¯1 1 ¯1 0 ¯1 ¯1 1 ¯1 0 ¯1 ¯1 ¯1 1 0 0 1 1 0 ¯1 ¯1 1 1 0 0 1 0 1 ¯1 0 1 1 0 0 1 0 0 0 0 1 1 0 ¯1 ¯1 ¯1 ¯1 0 ¯1 1 0 1 ¯1 ¯1 1 ¯1 ¯1 0 0 0 0 0 ¯1 1 ¯1 ¯1 1 1 ¯1 0 ¯1 ¯1 ¯1 0

In [16]:
)save notebook8 -force

notebook8.dws saved Tue Dec 22 16:45:36 2020